In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from code.experiment_functions import *

In [2]:
diabetes = pd.read_csv('data/diabetes.csv')

In [3]:
# normalization 0-1
for i in diabetes.columns:
    diabetes[i] = (diabetes[i] - diabetes[i].min()) / (diabetes[i].max() - diabetes[i].min())

In [4]:
# split in test and train data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diabetes.drop('Outcome',axis=1), diabetes['Outcome'])

In [5]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()
y_train = y_train.values.tolist()
y_test = y_test.values.tolist()

In [6]:
y_train = [int(x) for x in y_train]
y_test =  [int(x) for x in y_test]

### Train and Test with Bias

In [7]:
epoch_errosHSGS, epoch_errosEncodingWeight, epoch_errosEncodingInput  = experiment_TRAIN(X_train, 
                                                                                         y_train,
                                                                                         lrParameter=0.2, 
                                                                                         n_epochs=40, 
                                                                                         trainingBias=True,
                                                                                         error_by_epoch = False,
                                                                                         trainingApproaches={'neuronio-classico', 'neuronio-classico-bin', 'hsgs','encoding-weight', 'encoding-input'})

In [11]:
with open('error_by_epoch.json', 'w+') as fp:
    json.dump({'epoch_errosHSGS': epoch_errosHSGS,
               'epoch_errosEncodingWeight': epoch_errosEncodingWeight, 
               'epoch_errosEncodingInput': epoch_errosEncodingInput},
              fp)

In [7]:
weightsEncodingWeight, weightsEncodingInput, weightsHSGS, weightsVectorsClassico, weightVectorsClassicoBin = experiment_TRAIN(X_train, 
                                                                                                             y_train,
                                                                                                             lrParameter=0.2, 
                                                                                                             n_epochs=40, 
                                                                                                             trainingBias=False,
                                                                                                             trainingApproaches={'neuronio-classico', 'neuronio-classico-bin', 'hsgs','encoding-weight', 'encoding-input'})


erro HSGS 8.3740234375
erro encoding weight 436.7216796875
erro encoding input 182.53125
erro Classico 21.800604746665947
erro classico Bin 150.1796233262912

erro HSGS 0.0
erro encoding weight 435.8046875
erro encoding input 181.357421875
erro Classico 11.836733653224373
erro classico Bin 145.7823554784469

erro HSGS 0.0
erro encoding weight 436.2998046875
erro encoding input 180.7294921875
erro Classico 11.83895421034015
erro classico Bin 145.7823554784469

erro HSGS 0.0
erro encoding weight 435.63671875
erro encoding input 181.5234375
erro Classico 11.839079572712958
erro classico Bin 145.7823554784469

erro HSGS 0.0
erro encoding weight 435.55859375
erro encoding input 181.75390625
erro Classico 11.839084109064341
erro classico Bin 145.7823554784469

erro HSGS 0.0
erro encoding weight 436.1025390625
erro encoding input 181.6953125
erro Classico 11.839084278300644
erro classico Bin 145.7823554784469

erro HSGS 0.0
erro encoding weight 435.732421875
erro encoding input 181.19140625


In [ ]:
weights = [weightsEncodingWeight, 
          weightsEncodingInput, 
          weightsHSGS, 
          weightsVectorsClassico, 
          weightVectorsClassicoBin]

In [ ]:
with open('best_weights.json', 'w+') as fp:
    json.dump(weights, fp)

#### Test

In [8]:
with open('results/version2/weights_biased_original.json') as json_file:
    weights = json.load(json_file)

In [10]:
resultsBIAS = experiment_TEST(X_test, 
                              y_test, 
                              weights[0], 
                              weights[1], 
                              weights[2], 
                              weights[3], 
                              weights[4], 
                              repeat=5,
                              bias=True, 
                              testingApproaches={'neuronio-classico', 'neuronio-classico-bin', 'hsgs','encoding-weight', 'encoding-input'})

erro HSGS 0.3645833333333333
erro encoding weight 0.3645833333333333
erro encoding input 0.484375
erro classico 0.2760416666666667
erro classico bin 0.3645833333333333
erro HSGS 0.3645833333333333
erro encoding weight 0.3645833333333333
erro encoding input 0.46875
erro classico 0.2760416666666667
erro classico bin 0.3645833333333333
erro HSGS 0.3645833333333333
erro encoding weight 0.359375
erro encoding input 0.5364583333333334
erro classico 0.2760416666666667
erro classico bin 0.3645833333333333
erro HSGS 0.3645833333333333
erro encoding weight 0.3645833333333333
erro encoding input 0.4895833333333333
erro classico 0.2760416666666667
erro classico bin 0.3645833333333333
erro HSGS 0.3645833333333333
erro encoding weight 0.3645833333333333
erro encoding input 0.4635416666666667
erro classico 0.2760416666666667
erro classico bin 0.3645833333333333
ERROS HSGS             0.3646
ERROS ENCODING WEIGHT  0.36356
ERROS ENCODING INPUT   0.48856
ERROS Classico         0.276
ERROS Classico Bin  

In [11]:
with open('results/version2/experiments_biased_original.json', 'w+') as fp:
    json.dump(resultsBIAS, fp)

In [ ]:
resultsBIAS

In [21]:
def readAndResults(i):
    with open('results/'+i) as f:
        data = json.load(f)

    erros = ['error_HSGS', 'error_encoding_weight', 'error_encoding_input', 'error_classic', 'error_classic_bin']
    new_data = {'error':[], 'model':[]}

    for erro in erros:
        new_data['error'] = new_data['error'] + data[erro] 
        new_data['model'] = new_data['model'] +  ( (erro+' ') * len(data[erro])).split(' ')[:-1]  

    new_data = pd.DataFrame(new_data)

    new_data.to_csv('results/'+i[:-5]+'_dataframe.csv', index=False)
    
    return new_data

In [22]:
readAndResults('HEALTH_ZERO.json')

,error,model
0,0.3646,error_HSGS
1,0.3646,error_HSGS
2,0.3646,error_HSGS
3,0.3646,error_HSGS
4,0.3646,error_HSGS
...,...,...
195,0.3646,error_classic_bin
196,0.3646,error_classic_bin
197,0.3646,error_classic_bin
198,0.3646,error_classic_bin


In [5]:
df = pd.read_csv('results/HEALTH_ZERO_dataframe.csv')

In [10]:
np.mean(df[df.model == 'error_encoding_weight'])

error    0.36291
dtype: float64

In [11]:
np.mean(df[df.model == 'error_encoding_input'])

error    0.480335
dtype: float64

In [12]:
df2 = pd.read_csv('results/results_withoutbias_dataframe.csv')

In [13]:
np.mean(df2[df2.model == 'error_encoding_weight'])

error    0.573043
dtype: float64

In [14]:
np.mean(df2[df2.model == 'error_encoding_input'])

error    0.493495
dtype: float64